In [5]:
# Imports and Setup
import os
import math
import random
import logging
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr  # Correct import
from tqdm import tqdm, TqdmWarning

# Import custom modules
from EpiGNN.data_merge import merge_and_save
from EpiGNN.epignn_model import EpiGNN

# Suppress specific warnings
warnings.filterwarnings("ignore", category=TqdmWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="torch.optim.lr_scheduler")
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

# Set plotting style for publication-quality figures
plt.style.use('seaborn-v0_8-paper')
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'axes.titlesize': 16,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'font.size': 12,
    'figure.dpi': 300  # High resolution (DPI) for publication
})

sns.set(style="whitegrid")
plt.rcParams.update({"figure.max_open_warning": 0})

# Initialize logging
project_root = os.path.abspath(os.path.join(os.getcwd(), "../GNN_spatiotemporal_project"))  # Adjust as needed
log_file = os.path.join(project_root, "experiment.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ],
)

# Set random seeds for reproducibility
def seed_everything(seed=123):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everything()

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")


[2025-01-21 18:29:22] INFO: Using device: cuda


In [6]:
# Utility Functions

def compute_geographic_adjacency(regions, latitudes, longitudes, threshold=300):
    """
    Compute a static adjacency matrix based on geographic distances.

    Parameters:
    - regions (list): List of region names.
    - latitudes (list): List of latitudes corresponding to regions.
    - longitudes (list): List of longitudes corresponding to regions.
    - threshold (float): Distance threshold to determine adjacency.

    Returns:
    - adj_mat (torch.Tensor): Adjacency matrix (num_nodes, num_nodes).
    """
    from math import radians, sin, cos, asin, sqrt
    from scipy.spatial.distance import pdist, squareform

    def haversine_distance(u, v):
        lat1, lon1 = map(radians, [u[0], u[1]])
        lat2, lon2 = map(radians, [v[0], v[1]])
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = (sin(dlat / 2) ** 2) + cos(lat1) * cos(lat2) * (sin(dlon / 2) ** 2)
        c = 2 * asin(sqrt(a))
        return c * 6371  # Radius of Earth in kilometers

    coords = np.column_stack((latitudes, longitudes))
    dist_mat = squareform(pdist(coords, metric=haversine_distance))
    adj_mat = (dist_mat <= threshold).astype(np.float32)
    np.fill_diagonal(adj_mat, 1.0)  # Ensure self-connections
    return torch.tensor(adj_mat, dtype=torch.float32)


def getLaplaceMat(bs, m, adj):
    """
    Computes the Laplacian matrix.

    Parameters:
    - bs (int): Batch size.
    - m (int): Number of nodes.
    - adj (torch.Tensor): Adjacency matrix (batch_size, m, m).

    Returns:
    - laplace (torch.Tensor): Normalized Laplacian matrix (batch_size, m, m).
    """
    eye = torch.eye(m, device=adj.device).unsqueeze(0).repeat(bs, 1, 1)
    adj_bin = (adj > 0).float()
    deg = torch.sum(adj_bin, dim=2)  # (batch_size, m)
    deg_inv = 1.0 / (deg + 1e-12)  # Avoid division by zero
    deg_inv_mat = eye * deg_inv.unsqueeze(2)  # (batch_size, m, m)
    laplace = torch.bmm(deg_inv_mat, adj_bin)  # (batch_size, m, m)
    return laplace


In [7]:
# Dataset Class

class DailyDataset(Dataset):
    """
    Dataset class for daily COVID-19 data using a sliding-window approach.

    - X: shape (num_timesteps_input, num_nodes, num_features)
    - Y: shape (num_timesteps_output, num_nodes)
    """
    def __init__(self, data: pd.DataFrame, num_timesteps_input=20, num_timesteps_output=7, scaler=None):
        super(DailyDataset, self).__init__()
        self.data = data.copy()
        self.num_timesteps_input = num_timesteps_input
        self.num_timesteps_output = num_timesteps_output

        # Region indexing
        self.regions = self.data["areaName"].unique()
        self.num_nodes = len(self.regions)
        self.region_to_idx = {r: i for i, r in enumerate(self.regions)}
        self.data["region_idx"] = self.data["areaName"].map(self.region_to_idx)

        # Relevant features
        self.features = ["new_confirmed", "new_deceased", "newAdmissions", "hospitalCases", "covidOccupiedMVBeds"]

        # Pivot: index=date, columns=region_idx, values=features => shape (#days, #nodes, #features)
        pivoted = self.data.pivot(index="date", columns="region_idx", values=self.features)
        pivoted.ffill(inplace=True)  # Forward fill
        pivoted.fillna(0, inplace=True)

        self.num_days = pivoted.shape[0]
        self.num_features = len(self.features)
        self.feature_array = pivoted.values.reshape(self.num_days, self.num_nodes, self.num_features)

        self.scaler = scaler
        if self.scaler is not None:
            arr_2d = self.feature_array.reshape(-1, self.num_features)
            arr_2d = self.scaler.fit_transform(arr_2d)
            self.feature_array = arr_2d.reshape(self.num_days, self.num_nodes, self.num_features)

    def __len__(self):
        return self.num_days - self.num_timesteps_input - self.num_timesteps_output + 1

    def __getitem__(self, idx):
        X = self.feature_array[idx : idx + self.num_timesteps_input]  # (T_in, num_nodes, num_feats)
        Y = self.feature_array[idx + self.num_timesteps_input : idx + self.num_timesteps_input + self.num_timesteps_output, :, 4]  # covidOccupiedMVBeds
        return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)


In [3]:
# Jupyter Notebook Cell 3: Data Loading and Preprocessing

# Define paths
raw_csv_path = os.path.join(project_root, "data", "raw", "merged_nhs_covid_data.csv")
processed_csv_path = os.path.join(project_root, "data", "processed", "daily_nhs_covid_data.csv")

# Merge and preprocess data
logging.info("Starting data merging and preprocessing...")
df_processed = merge_and_save(raw_csv_path, processed_csv_path)

# Display first few rows
df_processed.head()


[2025-01-21 18:28:30] INFO: Starting data merging and preprocessing...
[2025-01-21 18:28:30] INFO: Data loaded and coordinates assigned. Sorted by areaName and date.
[2025-01-21 18:28:30] INFO: Processed daily data saved -> /home/toor/Projects/GNN_spatiotemporal_project/data/processed/daily_nhs_covid_data.csv


,areaName,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,latitude,longitude
0,East of England,2020-04-01,0.0,1400,833.0,167,334.0,75.0,2938.0,455.0,6235410,151336,52.1766,0.425889
1,East of England,2020-04-02,119.0,1584,841.0,184,372.0,71.0,3310.0,526.0,6235410,151336,52.1766,0.425889
2,East of England,2020-04-03,162.0,1776,914.0,192,350.0,85.0,3660.0,611.0,6235410,151336,52.1766,0.425889
3,East of England,2020-04-04,171.0,1939,988.0,163,268.0,70.0,3928.0,681.0,6235410,151336,52.1766,0.425889
4,East of England,2020-04-05,219.0,2159,1230.0,220,281.0,91.0,4209.0,772.0,6235410,151336,52.1766,0.425889


In [8]:
# Dataset Preparation

# Initialize scaler
scaler = StandardScaler()

# Create dataset
dataset = DailyDataset(
    data=df_processed,
    num_timesteps_input=20,
    num_timesteps_output=14,  # You can adjust based on experiments
    scaler=scaler
)

ds_len = len(dataset)
logging.info(f"Dataset length: {ds_len}")

if ds_len <= 0:
    logging.error("Dataset length is insufficient. Exiting.")
    raise ValueError("Insufficient data.")

# Define train/val/test splits
train_size = int(0.7 * ds_len)
val_size = int(0.15 * ds_len)
test_size = ds_len - train_size - val_size

train_idx = list(range(0, train_size))
val_idx = list(range(train_size, train_size + val_size))
test_idx = list(range(train_size + val_size, ds_len))

train_subset = Subset(dataset, train_idx)
val_subset = Subset(dataset, val_idx)
test_subset = Subset(dataset, test_idx)

logging.info(f"Train size: {len(train_subset)}, Val size: {len(val_subset)}, Test size: {len(test_subset)}")

# Initialize DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)


[2025-01-21 18:30:05] INFO: Dataset length: 862
[2025-01-21 18:30:05] INFO: Train size: 603, Val size: 129, Test size: 130


In [9]:
# Model Initialization

# Compute static adjacency matrix
logging.info("Computing static adjacency matrix based on geographic distances...")
regions = dataset.regions.tolist()
latitudes = [df_processed[df_processed["areaName"] == r]["latitude"].iloc[0] for r in regions]
longitudes = [df_processed[df_processed["areaName"] == r]["longitude"].iloc[0] for r in regions]
adj_static = compute_geographic_adjacency(regions, latitudes, longitudes, threshold=300).to(device)

logging.info("Static Adjacency Matrix:")
logging.info(adj_static.cpu().numpy())

# Visualize static adjacency graph
logging.info("Saving static adjacency graph figure...")
fig_dir = os.path.join(project_root, "figures", "static_adjacency")
os.makedirs(fig_dir, exist_ok=True)

A_np = adj_static.cpu().numpy()
G = nx.from_numpy_array(A_np)
mapping = {i: r for i, r in enumerate(regions)}
G = nx.relabel_nodes(G, mapping)
pos = {r: (longitudes[i], latitudes[i]) for i, r in enumerate(regions)}

plt.figure(figsize=(10, 8))
nx.draw_networkx(
    G,
    pos,
    with_labels=True,
    node_size=700,
    node_color="lightblue",
    edge_color="gray",
    font_size=10
)
plt.title("Static Adjacency (Geographic)", fontsize=12)
plt.axis("off")
out_figpath = os.path.join(fig_dir, "geographic_adjacency_graph_static.png")
plt.savefig(out_figpath, dpi=300, bbox_inches="tight")
plt.close()
logging.info(f"Saved adjacency figure -> {out_figpath}")

# Define experiment parameters
horizons = [3, 7, 14]
adjacency_types = ["static", "dynamic", "hybrid"]
experiment_id = 1
summary_metrics = []


[2025-01-21 18:30:13] INFO: Computing static adjacency matrix based on geographic distances...
[2025-01-21 18:30:13] INFO: Static Adjacency Matrix:
[2025-01-21 18:30:13] INFO: [[1. 1. 1. 0. 1. 1. 0.]
 [1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 0. 0.]
 [1. 0. 1. 1. 1. 1. 0.]
 [1. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 0. 0. 1. 1.]]
[2025-01-21 18:30:13] INFO: Saving static adjacency graph figure...
[2025-01-21 18:30:13] INFO: Saved adjacency figure -> /home/toor/Projects/GNN_spatiotemporal_project/figures/static_adjacency/geographic_adjacency_graph_static.png


In [10]:
# Training and Evaluation Functions

def train_epoch(model, optimizer, criterion, train_loader, adjacency_type, adj_static):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_Y in train_loader:
        batch_X, batch_Y = batch_X.to(device), batch_Y.to(device)
        optimizer.zero_grad()

        bs_cur = batch_X.size(0)

        # Prepare adjacency matrix
        if adjacency_type == "static":
            adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
        elif adjacency_type == "dynamic":
            adj_input = torch.eye(dataset.num_nodes, device=device).unsqueeze(0).repeat(bs_cur, 1, 1)
        elif adjacency_type == "hybrid":
            adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
        else:
            raise ValueError("Invalid adjacency_type. Choose from 'static', 'dynamic', 'hybrid'.")

        # Forward pass
        pred = model(batch_X, adj_input, adjacency_type=adjacency_type)
        loss = criterion(pred, batch_Y)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        epoch_loss += loss.item()
    avg_loss = epoch_loss / len(train_loader) if len(train_loader) > 0 else 0.0
    return avg_loss


def validate_epoch(model, criterion, val_loader, adjacency_type, adj_static):
    model.eval()
    epoch_loss = 0.0
    all_val_preds = []
    all_val_actuals = []
    with torch.no_grad():
        for batch_Xv, batch_Yv in val_loader:
            batch_Xv, batch_Yv = batch_Xv.to(device), batch_Yv.to(device)
            bs_cur = batch_Xv.size(0)

            # Prepare adjacency matrix
            if adjacency_type == "static":
                adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
            elif adjacency_type == "dynamic":
                adj_input = torch.eye(dataset.num_nodes, device=device).unsqueeze(0).repeat(bs_cur, 1, 1)
            elif adjacency_type == "hybrid":
                adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
            else:
                raise ValueError("Invalid adjacency_type. Choose from 'static', 'dynamic', 'hybrid'.")

            # Forward pass
            pred = model(batch_Xv, adj_input, adjacency_type=adjacency_type)
            loss = criterion(pred, batch_Yv)
            epoch_loss += loss.item()

            all_val_preds.append(pred.cpu().numpy())
            all_val_actuals.append(batch_Yv.cpu().numpy())

    avg_loss = epoch_loss / len(val_loader) if len(val_loader) > 0 else 0.0

    # Compute R² scores
    if all_val_preds and all_val_actuals:
        val_preds_arr = np.concatenate(all_val_preds, axis=0)  # (N, horizon, m)
        val_acts_arr = np.concatenate(all_val_actuals, axis=0)  # (N, horizon, m)
        preds_2d = val_preds_arr.reshape(-1, dataset.num_nodes)
        acts_2d = val_acts_arr.reshape(-1, dataset.num_nodes)

        r2_vals = []
        for nd in range(dataset.num_nodes):
            if np.isnan(preds_2d[:, nd]).any() or np.isnan(acts_2d[:, nd]).any():
                r2_vals.append(float("nan"))
            else:
                r2_vals.append(r2_score(acts_2d[:, nd], preds_2d[:, nd]))
    else:
        r2_vals = [float("nan")] * dataset.num_nodes

    return avg_loss, r2_vals


def test_model(model, criterion, test_loader, adjacency_type, adj_static, scaler):
    model.eval()
    test_loss = 0.0
    test_preds = []
    test_acts = []
    with torch.no_grad():
        for batch_Xt, batch_Yt in test_loader:
            batch_Xt, batch_Yt = batch_Xt.to(device), batch_Yt.to(device)
            bs_cur = batch_Xt.size(0)

            # Prepare adjacency matrix
            if adjacency_type == "static":
                adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
            elif adjacency_type == "dynamic":
                adj_input = torch.eye(dataset.num_nodes, device=device).unsqueeze(0).repeat(bs_cur, 1, 1)
            elif adjacency_type == "hybrid":
                adj_input = adj_static.unsqueeze(0).repeat(bs_cur, 1, 1).to(device)
            else:
                raise ValueError("Invalid adjacency_type. Choose from 'static', 'dynamic', 'hybrid'.")

            # Forward pass
            predt = model(batch_Xt, adj_input, adjacency_type=adjacency_type)
            tloss = criterion(predt, batch_Yt)
            test_loss += tloss.item()

            test_preds.append(predt.cpu().numpy())
            test_acts.append(batch_Yt.cpu().numpy())

    avg_test_loss = test_loss / len(test_loader) if len(test_loader) > 0 else float("nan")

    # Combine Predictions
    if test_preds and test_acts:
        preds_arr = np.concatenate(test_preds, axis=0)  # (N, horizon, m)
        acts_arr = np.concatenate(test_acts, axis=0)    # (N, horizon, m)
    else:
        preds_arr = np.array([])
        acts_arr = np.array([])

    # Inverse Transform
    if preds_arr.size > 0 and scaler is not None:
        sc_covid = scaler.scale_[4]
        mn_covid = scaler.mean_[4]
        preds_arr = preds_arr * sc_covid + mn_covid
        acts_arr = acts_arr * sc_covid + mn_covid

    # Compute Final Metrics
    if preds_arr.size > 0:
        preds_2d = preds_arr.reshape(-1, dataset.num_nodes)
        acts_2d = acts_arr.reshape(-1, dataset.num_nodes)

        mae_per_node = mean_absolute_error(acts_2d, preds_2d, multioutput="raw_values")
        mse_per_node = mean_squared_error(acts_2d, preds_2d, multioutput="raw_values")
        rmse_per_node = np.sqrt(mse_per_node)
        r2_per_node = r2_score(acts_2d, preds_2d, multioutput="raw_values")

        pcc_per_node = []
        for i in range(dataset.num_nodes):
            if np.std(acts_2d[:, i]) < 1e-6 or np.std(preds_2d[:, i]) < 1e-6:
                pcc_per_node.append(0.0)
            else:
                pcc_val, _ = pearsonr(acts_2d[:, i], preds_2d[:, i])
                if np.isnan(pcc_val):
                    pcc_val = 0.0
                pcc_per_node.append(pcc_val)
    else:
        mae_per_node = [float("nan")] * dataset.num_nodes
        rmse_per_node = [float("nan")] * dataset.num_nodes
        r2_per_node = [float("nan")] * dataset.num_nodes
        pcc_per_node = [float("nan")] * dataset.num_nodes

    return avg_test_loss, mae_per_node, rmse_per_node, r2_per_node, pcc_per_node


In [11]:
# Running Experiments

for horizon in horizons:
    logging.info(f"\n=== Starting experiments for horizon={horizon} ===")
    # Update the dataset's output horizon if needed
    dataset.num_timesteps_output = horizon

    for adjacency_type in adjacency_types:
        logging.info(f"\n--- Experiment {experiment_id}: Adjacency Type = {adjacency_type}, Horizon = {horizon} ---")
        
        # Initialize model
        model = EpiGNN(
            num_nodes=dataset.num_nodes,
            num_features=dataset.num_features,
            num_timesteps_input=dataset.num_timesteps_input,
            num_timesteps_output=horizon,
            k=8,
            hidA=32,
            hidR=40,
            hidP=1,
            n_layer=3,
            num_heads=4,
            dropout=0.5,
            device=device
        ).to(device)

        optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=5e-4)
        criterion = nn.MSELoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

        best_val_loss = float("inf")
        patience_counter = 0
        train_losses = []
        val_losses = []

        # Training loop
        for epoch in range(1, 1001):  # NUM_EPOCHS = 1000
            train_loss = train_epoch(model, optimizer, criterion, train_loader, adjacency_type, adj_static)
            val_loss, r2_vals = validate_epoch(model, criterion, val_loader, adjacency_type, adj_static)

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            scheduler.step(val_loss)

            logging.info(f"Epoch {epoch}/1000 - Adjacency: {adjacency_type} - "
                         f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | R² per node: {r2_vals}")

            # Check for improvement
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                ckpt_dir = os.path.join(project_root, "models", f"experiment{experiment_id}_{adjacency_type}_h{horizon}")
                os.makedirs(ckpt_dir, exist_ok=True)
                ckpt_path = os.path.join(ckpt_dir, "best_model.pth")
                torch.save(model.state_dict(), ckpt_path)
                logging.info(f"[BEST] Saved model -> {ckpt_path}")
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= 20:  # EARLY_STOPPING_PATIENCE = 20
                    logging.info("Early stopping triggered.")
                    break

        # Plot Train vs Val Loss
        plt.figure(figsize=(10, 6))
        sns.lineplot(x=range(1, len(train_losses)+1), y=train_losses, label="Train Loss")
        sns.lineplot(x=range(1, len(val_losses)+1), y=val_losses, label="Val Loss")
        plt.xlabel("Epoch")
        plt.ylabel("MSE Loss")
        plt.title(f"Experiment {experiment_id}, Adjacency={adjacency_type}, Horizon={horizon}")
        plt.grid(True)
        plt.legend()
        fig_dir = os.path.join(project_root, "figures", f"experiment{experiment_id}_{adjacency_type}_h{horizon}", "training_validation_loss")
        os.makedirs(fig_dir, exist_ok=True)
        fig_path = os.path.join(fig_dir, f"train_val_loss_experiment{experiment_id}_{adjacency_type}_h{horizon}.png")
        plt.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.close()
        logging.info(f"Saved loss curves -> {fig_path}")

        # Testing
        # Load best model
        ckpt_path = os.path.join(project_root, "models", f"experiment{experiment_id}_{adjacency_type}_h{horizon}", "best_model.pth")
        model.load_state_dict(torch.load(ckpt_path, map_location=device))
        model.eval()

        test_loss, mae_per_node, rmse_per_node, r2_per_node, pcc_per_node = test_model(
            model, criterion, test_loader, adjacency_type, adj_static, scaler
        )

        logging.info(f"Experiment {experiment_id}, Adjacency={adjacency_type}, Horizon={horizon} => Test MSE: {test_loss:.4f}")

        # Store Metrics
        for i, region in enumerate(regions):
            metrics = {
                "Experiment_ID": experiment_id,
                "Adjacency_Type": adjacency_type,
                "Horizon": horizon,
                "Region": region,
                "MAE": mae_per_node[i],
                "RMSE": rmse_per_node[i],
                "R2_Score": r2_per_node[i],
                "Pearson_Correlation": pcc_per_node[i],
            }
            summary_metrics.append(metrics)

        # Save Final Model
        final_model_dir = os.path.join(project_root, "models", f"experiment{experiment_id}_{adjacency_type}_h{horizon}")
        os.makedirs(final_model_dir, exist_ok=True)
        final_model_path = os.path.join(final_model_dir, "epignn_final_model.pth")
        torch.save(model.state_dict(), final_model_path)
        logging.info(f"Final model saved -> {final_model_path}")

        experiment_id += 1


[2025-01-21 18:30:31] INFO: 
=== Starting experiments for horizon=3 ===
[2025-01-21 18:30:31] INFO: 
--- Experiment 1: Adjacency Type = static, Horizon = 3 ---
[2025-01-21 18:30:33] INFO: Epoch 1/1000 - Adjacency: static - Train Loss: 0.6803 | Val Loss: 3.5627 | R² per node: [-0.7766076286332821, -1934.986201261059, -164.3217033030471, -29.06437422402599, -1.480405840009709, -67.23877133209312, -1139.667783998025]
[2025-01-21 18:30:33] INFO: [BEST] Saved model -> /home/toor/Projects/GNN_spatiotemporal_project/models/experiment1_static_h3/best_model.pth
[2025-01-21 18:30:33] INFO: Epoch 2/1000 - Adjacency: static - Train Loss: 0.1498 | Val Loss: 2.7880 | R² per node: [-0.39509937561844843, -2053.538332497925, -75.70680124660774, -9.680329046729321, -0.5045304944331395, -56.30662675742704, -855.3262682531763]
[2025-01-21 18:30:33] INFO: [BEST] Saved model -> /home/toor/Projects/GNN_spatiotemporal_project/models/experiment1_static_h3/best_model.pth
[2025-01-21 18:30:33] INFO: Epoch 3/1000

In [12]:
# Saving and Visualizing Results

if summary_metrics:
    logging.info("Saving summary metrics...")
    summary_df = pd.DataFrame(summary_metrics)
    summary_metrics_path = os.path.join(project_root, "report", "metrics", "summary_metrics.csv")
    os.makedirs(os.path.dirname(summary_metrics_path), exist_ok=True)
    summary_df.to_csv(summary_metrics_path, index=False)
    logging.info(f"Saved summary metrics -> {summary_metrics_path}")

    # Pivot table for easier analysis
    summary_pivot = summary_df.pivot_table(
        index=["Experiment_ID", "Adjacency_Type", "Horizon"],
        columns="Region",
        values=["MAE", "RMSE", "R2_Score", "Pearson_Correlation"]
    ).reset_index()

    pivot_path = os.path.join(project_root, "report", "metrics", "summary_metrics_pivot.csv")
    summary_pivot.to_csv(pivot_path, index=False)
    logging.info(f"Saved pivoted summary metrics -> {pivot_path}")

    # Display the summary pivot
    display(summary_pivot)

    # Plotting R² scores for each experiment
    plt.figure(figsize=(12, 8))
    for idx, row in summary_pivot.iterrows():
        # Extract R2_Score columns
        r2_scores = row.filter(regex='R2_Score').values
        plt.plot(range(dataset.num_nodes), r2_scores, label=f"Exp {row['Experiment_ID']} - {row['Adjacency_Type']} - H={row['Horizon']}")

    plt.xlabel("Region Index")
    plt.ylabel("R² Score")
    plt.title("R² Scores Across Regions for All Experiments")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    r2_fig_path = os.path.join(project_root, "figures", "R2_Scores_All_Experiments.png")
    plt.savefig(r2_fig_path, dpi=300, bbox_inches="tight")
    plt.close()
    logging.info(f"Saved R² scores plot -> {r2_fig_path}")

else:
    logging.warning("No metrics collected. Possibly no data or skip conditions were triggered.")


[2025-01-21 18:32:29] INFO: Saving summary metrics...
[2025-01-21 18:32:29] INFO: Saved summary metrics -> /home/toor/Projects/GNN_spatiotemporal_project/report/metrics/summary_metrics.csv
[2025-01-21 18:32:29] INFO: Saved pivoted summary metrics -> /home/toor/Projects/GNN_spatiotemporal_project/report/metrics/summary_metrics_pivot.csv


Experiment_ID Adjacency_Type Horizon             MAE              \
Region                                      East of England      London   
0                  1         static       3      400.864502   53.169640   
1                  2        dynamic       3      455.903564   56.175747   
2                  3         hybrid       3      261.887268   36.029663   
3                  4         static       7      374.882507   48.354160   
4                  5        dynamic       7      350.498474   51.172108   
5                  6         hybrid       7      270.753601   74.751221   
6                  7         static      14      475.789093   81.130615   
7                  8        dynamic      14      712.360840  111.044838   
8                  9         hybrid      14      457.934174   61.979706   

                                                                     \
Region    Midlands North East and Yorkshire  North West  South East   
0        90.318077                73.351532  169.137894   31.086168   
1        39.251194                40.301373  152.586700   55.301769   
2        69.749924                40.422863   88.413620   91.918427   
3        63.927673                42.700542  188.571350   63.029427   
4        43.815174                46.539074  144.971069   53.770432   
5       101.637466                65.500671  105.067505  120.156052   
6        48.264317                57.271732  260.767334   68.014053   
7       106.242104                72.966202  400.052582  165.471542   
8       148.109756                54.170204  156.825989  168.237900   

                   ...   R2_Score                                    RMSE  \
Region South West  ... North West  South East  South West East of England   
0       66.326225  ...   0.809810   -6.000605 -368.075761      504.622620   
1       33.912563  ...   0.830847  -22.085635 -106.504013      675.480957   
2       37.628368  ...   0.951277  -55.144567 -180.909439      340.864746   
3       54.369957  ...   0.772885  -34.833104 -334.131495      529.291016   
4       51.249371  ...   0.861427  -29.373757 -231.533264      526.026855   
5       81.661873  ...   0.901054  -93.487727 -710.642314      375.147003   
6       53.878235  ...   0.528456  -36.363822 -339.862227      714.919067   
7       77.119850  ...  -0.019061 -191.411017 -732.794402     1017.626831   
8       65.782326  ...   0.804760 -197.024843 -431.374304      698.703857   

                                                                     \
Region      London    Midlands North East and Yorkshire  North West   
0        70.700661  108.621620                89.411430  205.378174   
1        71.888664   45.214993                49.982899  193.686630   
2        45.062958   91.408066                54.077755  103.950218   
3        63.534325   77.280136                54.898769  221.444763   
4        70.067818   51.908897                54.082279  172.974228   
5        91.159660  119.164604                81.529709  146.164276   
6       107.057175   62.716988                70.754715  317.023438   
7       147.850784  134.537445                91.253967  466.048279   
8        77.354065  164.514420                67.831657  203.992630   

                               
Region  South East South West  
0        37.991550  72.289474  
1        68.990608  39.014832  
2       107.590286  50.751030  
3        85.755196  67.423531  
4        78.952789  56.162479  
5       139.253387  98.250465  
6        87.854584  67.778801  
7       199.367126  99.447021  
8       202.254807  76.336929  

[9 rows x 31 columns]

[2025-01-21 18:32:30] INFO: Saved R² scores plot -> /home/toor/Projects/GNN_spatiotemporal_project/figures/R2_Scores_All_Experiments.png
